<a href="https://colab.research.google.com/github/pbarizza/ML-course-2025/blob/main/lesson03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
21-Jun-2025 (Summer Solstice)
Lesson 03
Logistic Regression
